# PageRank as an optimization problem

The goal of this notebook is to implement Stochastic Gradient Descent (SGD) and compare it to Gradient Descent (GD) for solving the PageRank optimization problem
$$
\min_{x \in \mathbb{R}^n} f(x) := \frac{1}{2}\Vert Mx - x\Vert^2 + \frac{\lambda}{2} \left(\sum_{i=1}^n x[i] - 1\right)^2 ,
$$
where $M = (1 - \alpha)A + \alpha/n$, $\alpha \in ]0,1]$ ($0.15$ in our implementation), and $A$ is the link matrix of a web. The theoretical properties of GD and SGD for quadratic problems were studied in the course and TD 3.

In [1]:
%reset -f
import os
import sys
import curses
import numpy as np
import numpy.matlib
import matplotlib
import matplotlib.pyplot as plt
import scipy
import scipy.sparse as scisparse
import pkgutil
from time import time
from importlib import reload
from matplotlib import interactive
interactive(True)
%matplotlib notebook

# Utility functions

In [2]:
def mMatrix(A,alpha = 0.15):
    ''' Creates the regularized version of A with alpha=0.15 '''
    n = A.shape[0]
    M = (1-alpha)*A+alpha/n
    return M

def scorePageRank(A):
    ''' Computes the score vector using a full eigenvalue decomposition '''
    n = A.shape[0]
    M = mMatrix(A)
    v, w = np.linalg.eig(M)
    i = np.where(np.abs(v-1) < 1e-5)
    score = w[:, i].reshape((n, 1))
    score /= score.sum()
    return score.real

def dotProd(x, y):
    ''' Computes the dot product between x and y '''
    return np.ndarray.item(np.inner(x.reshape(-1), y.reshape(-1)))

def MakeCol(y): return y.reshape(-1,1)

def computeEigHF(A, lam):
    ''' Computes the eigenvalue decomposition of the Hessian to get the convergence rate of GD with step-size gamma.
    Parameters:
    -----------
        A (np.matrix): the matrix of links
        lam (float): regularization parameter
    Returns:
    --------
        v (float): the eignevalues of HF
    '''
    n = A.shape[0]
    M = mMatrix(A)
    In = np.eye(n)
    e = np.matlib.ones((n, 1))
    HF = (In-M).transpose()*(In-M) + lam*e*e.transpose()
    v, w= np.linalg.eig(HF)
    return v

# Miniwebs

In [3]:
def createMiniweb1():
    ''' Creates a 4x4 miniweb matrix '''
    A = np.matrix([[0, 1/3., 1/3., 1/3.],
                   [0, 0, 1/2., 1/2.],
                   [1, 0, 0, 0],
                   [1/2., 0, 1/2., 0]]).transpose()
    return A


def createMiniweb2():
    ''' Creates a 5x5 miniweb matrix '''
    A = np.matrix([[0, 1, 0, 0, 0],
                   [1, 0, 0, 0, 0],
                   [0, 0, 0, 1, 0],
                   [0, 0, 1, 0, 0],
                   [0., 0, 1/2., 1/2., 0]]).transpose()
    return A


def createRandomWeb(n=15, density=0.15, sparse=True):
    ''' Creates a random web link matrix
    Parameters:
    -----------
        n (int, optional): size of the matrix
        density (float, optional) : density rate, should lie in [0,1]
        sparse (boolean, optional): True (sparse) False (full)
    Returns:
    --------
        A (np.matrix): the randomly generated link matrix
    Notes:
    ------
        The routine starts by choosing random connections, then add other
        random connections if some pages are not connected to any other, and
        then normalizes to compute the column-stochastic link matrix.
    '''
    A = np.matlib.zeros((n*n, 1))
    indices = np.random.choice(range(n*n), int(density*n*n), replace=False)
    A[indices] = 1
    A = A.reshape((n, n))
    nLinks = np.sum(A, 0)
    nonConnectedCols = np.arange(n)[np.asarray(nLinks).reshape(-1) == 0]
    nLinks = np.asarray(np.matlib.repmat(nLinks, n, 1))
    indices = np.random.choice(range(n), len(nonConnectedCols), replace=True)
    A[A != 0] = 1 / nLinks[A != 0]
    A[indices, nonConnectedCols] = 1
    if sparse is True:
        return scisparse.csr_matrix(A)
    else:
        return A 

# Gradient descent
Apply GD
$$ 
x_{k+1} = x_k - \gamma \nabla f(x_k)
$$
to solve the PageRank optimization problem. The gradient $\nabla f(x)$ must be computed efficiently as
$$
\nabla f(x) = x-z - (1-\alpha)A^\top(x-z) + \lambda(s-1)
$$
where $s=\sum_{i=1}^n x[i]$ and $z=(1-\alpha)Ax + \alpha s/n$.

In [4]:
## Insert your code here.

# Stochastic Gradient Descent
The PageRank optimization objective can be equivalently written as
$$
f(x) = \frac{1}{2(n+1)}\Vert \mathcal{A} x - b\Vert^2 .
$$
where $\mathcal{A}=\begin{pmatrix} I - M \\ \sqrt{\lambda} E \end{pmatrix}$ and $b=(0,\cdots,0,\sqrt{\lambda})^\top$, $E=(1,\ldots,1)$. Computing $\nabla f(x)$ is prohibitive for large $k$ (cost/iter is $O(n^2)$ for dense and $O(density*n^2)$ for sparse graph). This finite sum structure (empirical risk) is amenable to SGD
$$ 
x_{k+1} = x_k - \gamma \nabla f_{i_k}(x_k)
$$
where $i_k$ is drawn uniformly at random in $\{1,\ldots,n+1\}$, and
$$
\nabla f_i(x) = 
\begin{cases}
\left((1-\alpha)a_i +  \frac{\alpha}{n} - e_i\right) \left((1-\alpha)\langle a_i,x \rangle + \frac{\alpha}{n} \sum_{j=1}^n x[j] - x[i]\right) & i=1,\ldots,n , \\
\lambda \left(\sum_{j=1}^n x[j] - 1\right) & i=n+1 ,
\end{cases}
$$
with $a_i = A[i,:]^\top$. Computing $\nabla f_i(x)$ costs $O(n)$ per iteration. See also the comparison of running times is the last section of this notebook.

In [5]:
## Insert your code here.

# Test and compare SGD and GD for different miniwebs
* Monitor the residual error decay. 
* Test different step-sizes, and compare with the theory (in particular plot in log domain to illustrate the linear rate). 

In [6]:
## Insert your code here.

# Compare running times
Compare running times for increasingly large graphs with a fixed sparsity rate and for different implementations of PageRank (GD and SGD with sparse vs dense $A$).

In [7]:
## Insert your code here.